In [2]:
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
import dotenv
import os

dotenv.load_dotenv()

True

In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

index_name = "showcase-index"

pinecone_store = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

prompt_template = """
Answer the question based on the provided context.
If the context doesn't contain the answer, say that you don't know.

Context: {context}
Question: {question}

Answer: """

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

/Users/tomas/tool_box/venv/lib/python3.13/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pinecone_store.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT}
)

In [5]:
def query_knowledge_base(question: str) -> str:
    return qa_chain.invoke({"query": question})

question = "Are male cats more likely to be left-pawed?"
result = query_knowledge_base(question)
print(result['result'])

Yes, male cats are more likely to be left-pawed.


In [6]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_anthropic import ChatAnthropic
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver

# Initialize the models
router_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)
general_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
index_name = "showcase-index"

pinecone_store = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# Create the prompt template for the cat knowledge base
prompt_template = """
Answer the question based on the provided context.
If the context doesn't contain the answer, say that you don't know.

Context: {context}
Question: {question}

Answer: """

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Create the cat knowledge QA chain
cat_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=pinecone_store.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT}
)

# Define the routing function
def route_question(state: MessagesState) -> Literal["cat_knowledge", "general_knowledge"]:
    messages = state['messages']
    question = messages[-1].content
    print(f"Evaluating question: {question}")
    
    # Ask the LLM to evaluate if this is a cat-related question
    evaluation_prompt = f"""Determine if the following question is about cats/felines. 
    Question: {question}
    Reply with just 'cat_knowledge' or 'general_knowledge'."""
    
    response = router_model.invoke([HumanMessage(content=evaluation_prompt)])
    route = "cat_knowledge" if "cat_knowledge" in response.content.lower() else "general_knowledge"
    print(f"Routing to: {route}")
    return route

def handle_cat_question(state: MessagesState):
    question = state['messages'][-1].content
    print(f"Handling cat question: {question}")
    result = cat_qa.invoke({"query": question})
    return {"messages": [HumanMessage(content=str(result))]}

def handle_general_question(state: MessagesState):
    question = state['messages'][-1].content
    print(f"Handling general question: {question}")
    response = general_model.invoke([HumanMessage(content=question)])
    return {"messages": [response]}


In [7]:
# Create the graph
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("cat_knowledge", handle_cat_question)
workflow.add_node("general_knowledge", handle_general_question)

# Add conditional edges from start
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "cat_knowledge": "cat_knowledge",
        "general_knowledge": "general_knowledge"
    }
)

# Add edges to end
workflow.add_edge("cat_knowledge", END)
workflow.add_edge("general_knowledge", END)


In [8]:
# Initialize memory
checkpointer = MemorySaver()

# Compile the graph
app = workflow.compile(checkpointer=checkpointer)

print("\n=== Testing Cat Question ===")
final_state = app.invoke(
    {"messages": [HumanMessage(content="What do cats like to eat?")]},
    config={"configurable": {"thread_id": "cat_test"}}
)
print(final_state["messages"][-1].content)

print("\n=== Testing General Question ===")
final_state = app.invoke(
    {"messages": [HumanMessage(content="What is the capital of France?")]},
    config={"configurable": {"thread_id": "general_test"}}
)
print(final_state["messages"][-1].content)


=== Testing Cat Question ===
Evaluating question: What do cats like to eat?
Routing to: cat_knowledge
Handling cat question: What do cats like to eat?
{'query': 'What do cats like to eat?', 'result': 'Cats like to eat foods such as cat food, mice, and other small animals.'}

=== Testing General Question ===
Evaluating question: What is the capital of France?
Routing to: general_knowledge
Handling general question: What is the capital of France?
The capital of France is Paris.
